## 은행 컨트랙을 프로그램하여 아래 기능을 실행하도록 하세요.
* 입금액을 매개변수로 넣어서 입금하세요. amount가 실제 입금액과 일치하지 않으면 오류.

	function deposit(uint amount)

* 전액 인출 함수를 구현하세요. 인출 금액을 정하지 않아도 되므로 함수의 인자가 없다.

	function widthdrawAll()

* 컨트랙 잔고확인 함수 (this를 이용한 잔고, 상태변수 잔고 (this 잔고가 맞는지 확인하는 용도)

	function getBalance() public view returns(uint, uint)

* 다른 계정으로 계좌이체 함수

	function forwardTo(address payable _receiver) public payable

* fallback()을 입금가능하도록 구현하세요. fallback이 호출되면 이벤트가 발생하여 전송자와 금액을 출력하세요.

이벤트 함수명은 PrintLog(address from, uint amount)



주피터 노트북에서 컴파일, 배포한 후, 아래 기능을 실행하여 그 결과를 출력하세요.

ganache 8345 띄우고, 노드로 하세요.

- 1. 입금 11111 wei, 222 wei를 하고 컨트랙잔고 11333 wei 출력

- 2. 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력

- 3. 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)

## ganache

## 1. 개발

In [7]:
%%writefile src/MyBank.sol
pragma solidity 0.6.4;

contract MyBank {
    address payable owner;
    uint balance;
    constructor() public {
        owner = msg.sender;
        balance = address(this).balance;
    }
    
    function deposit(uint amount) public payable {
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public {
        uint amount = balance;
        balance -= amount;
        owner.transfer(amount);
    }
    function getBalance() public view returns(uint, uint) {
        return (address(this).balance, balance);
    }
    function forwardTo(address payable _receiver) public payable {
        uint amount = 333;
        balance -= amount;
        _receiver.transfer(amount);
    }
}

Overwriting src/MyBank.sol


## 2.컴파일

In [8]:
!solc --abi --bin --gas src/MyBank.sol


======= src/MyBank.sol:MyBank =======
Gas estimation:
construction:
   41047 + 116600 = 157647
external:
   deposit(uint256):	21103
   forwardTo(address):	infinite
   getBalance():	1027
   withdrawAll():	infinite
Binary:
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555047600181905550610247806100676000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad8814610076578063853828b6146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf610170565b005b6100fd600480360360208110156100e757600080fd5b81019080803590602001909291905050506101f2565b005b60008047600154915091509091565b600061014d90508060016000828254

## 3.배포

In [13]:
!node src/getMyAddr.js

[
  '0x5723A0c574E487d8DA6FDAf9F66b65f7fe7D8a42',
  '0x27DEf991DE494718AEFCBFc829FA001643E48765',
  '0x3254D17544E9136FD903578d6836c9a924b0722B',
  '0xc81d5Dd8F9Fb8F5140dEda82977102A76a5beDb1',
  '0x056f31bE115b0886A9607F773b81438c68f5a6c6',
  '0xd9f57b3d8d613456aDD91Ba3A767f1f5dFF92574',
  '0xEA22778AF5c5cBCe793082E5Af43A870bEFf9445',
  '0x1A2b6337ef60eb9CFEadb66aD736A68d194CEBf4',
  '0xDc9BEeB1C8a1e791E6cecc44c434ec3972413CA0',
  '0xdE4438C8E3cEB131d2B0c212Ac4A371ef80D066A'
]


In [14]:
%%writefile src/MyBankDeploy.js
var Web3 = require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin = "608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555047600181905550610247806100676000396000f3fe60806040526004361061003f5760003560e01c806312065fe01461004457806327d8ad8814610076578063853828b6146100ba578063b6b55f25146100d1575b600080fd5b34801561005057600080fd5b506100596100ff565b604051808381526020018281526020019250505060405180910390f35b6100b86004803603602081101561008c57600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061010e565b005b3480156100c657600080fd5b506100cf610170565b005b6100fd600480360360208110156100e757600080fd5b81019080803590602001909291905050506101f2565b005b60008047600154915091509091565b600061014d9050806001600082825403925050819055508173ffffffffffffffffffffffffffffffffffffffff166108fc829081150290604051600060405180830381858888f1935050505015801561016b573d6000803e3d6000fd5b505050565b60006001549050806001600082825403925050819055506000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff166108fc829081150290604051600060405180830381858888f193505050501580156101ee573d6000803e3d6000fd5b5050565b8034146101fe57600080fd5b806001600082825401925050819055505056fea264697066735822122039009b4f1f622ac28bb0f72f7543f30f04991eb9df2008362d5162ad93d2256364736f6c63430006040033";
var _contract = new web3.eth.Contract(_abi);

_contract
    .deploy({ data: "0x"+_bin })
    .send({
     from: '0x5723A0c574E487d8DA6FDAf9F66b65f7fe7D8a42',
     gas: '1000000'
    })
    .then(function(newContractInstance){
        console.log("contract address: "+newContractInstance.options.address)
    });

Overwriting src/MyBankDeploy.js


In [15]:
!node src/MyBankDeploy.js

contract address: 0xD2D284A449032906FE39c483c88CfB0a314cEE99


## 4. 사용

### 1) 입금 11111 wei, 222 wei를 하고 잔고 11333 wei 출력

In [16]:
%%writefile src/MyBankUse1.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abi, "0xD2D284A449032906FE39c483c88CfB0a314cEE99");
var coinbase = "0x5723A0c574E487d8DA6FDAf9F66b65f7fe7D8a42";

bank.methods.deposit(11111).send({from:coinbase, gas:1000000, value:11111});
bank.methods.deposit(222).send({from:coinbase, gas:1000000, value:222});

Writing src/MyBankUse1.js


In [17]:
%%writefile src/MyBankUse-chk-contract.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abi, "0xD2D284A449032906FE39c483c88CfB0a314cEE99");
var coinbase = "0x5723A0c574E487d8DA6FDAf9F66b65f7fe7D8a42";

bank.methods.getBalance().call().then(console.log);

Writing src/MyBankUse-chk-contract.js


In [18]:
!node src/MyBankUse1.js

In [19]:
!node src/MyBankUse-chk-contract.js

Result { '0': '11333', '1': '11333' }


## 2) 자신의 2번째 계정으로 계좌이체 333 wei하고 컨트랙 잔고 11000 wei 출력, 자신의 2번째 계정 잔고 증가분 (+333 wei) 출력

In [20]:
%%writefile src/MyBankUse2-chk.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abi, "0xD2D284A449032906FE39c483c88CfB0a314cEE99");
var coinbase = "0x5723A0c574E487d8DA6FDAf9F66b65f7fe7D8a42";
var receiver = "0x27DEf991DE494718AEFCBFc829FA001643E48765";

web3.eth.getBalance(receiver).then(function(ret) {
    console.log("receiver balance: "+ret);
})

Writing src/MyBankUse2-chk.js


In [21]:
!node src/MyBankUse2-chk.js

receiver balance: 1000000000000000000000


In [22]:
%%writefile src/MyBankUse2.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abi, "0xD2D284A449032906FE39c483c88CfB0a314cEE99");
var coinbase = "0x5723A0c574E487d8DA6FDAf9F66b65f7fe7D8a42";
var receiver = "0x27DEf991DE494718AEFCBFc829FA001643E48765";

bank.methods.forwardTo(receiver).send({from:coinbase, gas:1000000});

Writing src/MyBankUse2.js


In [23]:
!node src/MyBankUse2.js

In [24]:
!node src/MyBankUse2-chk.js

receiver balance: 1000000000000000000333


In [25]:
!node src/MyBankUse-chk-contract.js

Result { '0': '11000', '1': '11000' }


## 3) 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)

In [26]:
%%writefile src/MyBankUse3-chk.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abi, "0xD2D284A449032906FE39c483c88CfB0a314cEE99");
var coinbase = "0x5723A0c574E487d8DA6FDAf9F66b65f7fe7D8a42";

web3.eth.getBalance(coinbase).then(function(ret) {
    console.log("coinbase balance: "+ret);
})

Writing src/MyBankUse3-chk.js


In [28]:
!node src/MyBankUse3-chk.js

coinbase balance: 999999378493999988667


In [31]:
%%writefile src/MyBankUse3.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdrawAll","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var bank = new web3.eth.Contract(_abi, "0xD2D284A449032906FE39c483c88CfB0a314cEE99");
var coinbase = "0x5723A0c574E487d8DA6FDAf9F66b65f7fe7D8a42";

bank.methods.withdrawAll().send({from:coinbase, gas:1000000});

Overwriting src/MyBankUse3.js


In [32]:
!node src/MyBankUse3.js

In [33]:
!node src/MyBankUse3-chk.js

coinbase balance: 999999317261999999667


In [34]:
!node src/MyBankUse-chk-contract.js

Result { '0': '0', '1': '0' }
